In [100]:
from pynucastro.screening import chugunov_2009
from keras_network import ScreeningFactorData, ScreeningFactorNetwork

import pynucastro as pyna
import numpy as np

In [101]:
data = ScreeningFactorData(screen_func=chugunov_2009, threshold=1.01, size=2*10**5, seed=1)

In [102]:
data.x

{'train': array([[  8.53546487,   7.80185479, 221.65891599, ...,  78.26204045,
          51.73053441,  95.20666509],
        [  7.43247884,   7.79459779,   4.48380342, ...,  47.1063948 ,
           7.44583828,  12.20054858],
        [  7.93549436,   5.6933058 ,  53.11888764, ..., 152.43070134,
          12.26132038,  24.27125047],
        ...,
        [  7.61002059,   6.07059977,  12.6067145 , ...,  68.82459955,
          15.352061  ,  36.64394483],
        [  7.87854679,   5.41044767,  55.13586294, ...,  67.63059194,
          26.14423804,  48.29893955],
        [  7.62821743,   4.03969554,   1.65776815, ...,  46.5980737 ,
         103.70886851, 209.54656393]]),
 'validate': array([[  8.81904723,   4.75623397, 172.87842454, ...,  18.85972751,
          55.49373416, 120.3875333 ],
        [  7.13127339,   5.87757719, 201.95006443, ...,  40.81004313,
          35.84123381,  54.07940171],
        [  8.23891941,   4.14795463,  10.25423572, ...,  12.38811751,
         100.08071722, 209.881

In [103]:
data.frac_pos

0.0021516666666666667

In [104]:
network = ScreeningFactorNetwork(data=data, seed=1)

In [105]:
class_weights = {k: v for k,v in network.class_weight.items()}
class_weights[0] *= 40
display(
    class_weights
)

{0: 100.21563063190966, 1: 232.37800154918668}

In [106]:
import keras
network.callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='fn',
        verbose=1,
        patience=25,
        mode='min',
        restore_best_weights=True
    )
]

In [107]:
network.class_weight = class_weights
network.compile(learning_rate=1.05e-4)

In [108]:
history = network.fit_model(verbose=2)
network.score

Epoch 1/500
98/98 - 14s - 142ms/step - auc: 0.3772 - fn: 427.0000 - fp: 3028.0000 - loss: 14.4458 - precision: 0.0013 - recall: 0.0093 - tn: 196541.0000 - tp: 4.0000 - val_auc: 0.5632 - val_fn: 441.0000 - val_fp: 0.0000e+00 - val_loss: 0.0195 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 199559.0000 - val_tp: 0.0000e+00
Epoch 2/500
98/98 - 5s - 55ms/step - auc: 0.8146 - fn: 431.0000 - fp: 0.0000e+00 - loss: 2.8240 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 199569.0000 - tp: 0.0000e+00 - val_auc: 0.8769 - val_fn: 441.0000 - val_fp: 0.0000e+00 - val_loss: 0.0144 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_tn: 199559.0000 - val_tp: 0.0000e+00
Epoch 3/500
98/98 - 5s - 48ms/step - auc: 0.9045 - fn: 431.0000 - fp: 0.0000e+00 - loss: 2.2070 - precision: 0.0000e+00 - recall: 0.0000e+00 - tn: 199569.0000 - tp: 0.0000e+00 - val_auc: 0.9017 - val_fn: 441.0000 - val_fp: 0.0000e+00 - val_loss: 0.0125 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_

[0.0024360225070267916,
 0.9902230501174927,
 82.0,
 66.0,
 0.8362283110618591,
 0.8042959570884705,
 199515.0,
 337.0]

In [109]:
nuclei = [pyna.Nucleus("h1"),
          pyna.Nucleus("he4"),
          pyna.Nucleus("c12"),
          pyna.Nucleus("o16"),
          pyna.Nucleus("n14"),
          pyna.Nucleus("ca40")]

reactants = [pyna.Nucleus("c12"), pyna.Nucleus("he4")]

comp = pyna.Composition(nuclei=nuclei)
comp.set_solar_like()

scn_fac = pyna.make_screen_factors(*reactants)

In [110]:
dens, temp = 1.e5, 3e7

plasma = pyna.make_plasma_state(temp, dens, comp.get_molar())

In [111]:
chugunov_2009(plasma, scn_fac)

12.784589340763912

In [112]:
can_skip = network.predict(temp=temp, dens=dens, comp=comp, nuclei=reactants)
display(not can_skip)

True

In [113]:
T_, D_ = 10**np.linspace(8, 11), 10**np.linspace(4, 8)
T, D = np.meshgrid(T_, D_)

In [114]:
#network.predict(temp=T, dens=D, comp=comp, nuclei=reactants)

: 

load data using yt